In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
# read csv file:
handle=open('./SecilSH_turkce-veriseti-master/turkce_veriseti.csv',encoding="utf-8")
# Kayıt deseni: "Ana Kategori", Alt Kategori, Website Adresi,  DMOZ Başlığı, DMOZ Açıklaması, Website Başlığı, Website Meta Açıklaması, Website Meta Kelimeleri, "Website İçeriği"

In [3]:
categories_data= {} #dictionary
for line in handle:
    my_list= line.split(',')
    main_category= my_list[0]
    main_category= main_category[1:-1]
    content= my_list[-1]
    content= content[1:-1]
   
    if main_category not in categories_data.keys():
        categories_data[main_category]= []
    categories_data[main_category].append(content)
print(my_list[0])
print("done")


"Çocuklar_ve_Gençler"
done


In [4]:
import os.path
for cat in categories_data.keys():
    # create a new file:
    file_name= "./data/"+ cat+".txt" # indicates after the current folder: ./
    with open(file_name,"w",encoding="utf8") as f: # +: create if not exists
        for item in categories_data[cat]:
                item = item.replace(' "','')
                f.write(item+'\n')
    f.close()
           # print(item)


In [5]:
# Corpus reader:
import os
root = './data/'
from nltk.corpus.reader import CategorizedPlaintextCorpusReader
reader = CategorizedPlaintextCorpusReader(root, r'.*\.txt', cat_pattern=r'(\w+)/*', encoding='utf-8')

In [6]:
print(reader.categories())
print(reader.fileids())


['Alışveriş', 'Basın_ve_Yayın', 'Bilgisayar', 'Bilim', 'Ekonomi_ve_İş_Dünyası', 'Ev', 'Eğlence_ve_Yaşam', 'Kaynaklar', 'Kültür_ve_Sanat', 'Oyunlar', 'Sağlık', 'Spor', 'Toplum', 'Çocuklar_ve_Gençler']
['Alışveriş.txt', 'Basın_ve_Yayın.txt', 'Bilgisayar.txt', 'Bilim.txt', 'Ekonomi_ve_İş_Dünyası.txt', 'Ev.txt', 'Eğlence_ve_Yaşam.txt', 'Kaynaklar.txt', 'Kültür_ve_Sanat.txt', 'Oyunlar.txt', 'Sağlık.txt', 'Spor.txt', 'Toplum.txt', 'Çocuklar_ve_Gençler.txt']


In [7]:
def sent_tokenize_whole_tweets(text): # raw text --> whole tweets file content
    sents = text.split('\n')
    for s in sents:
        s= s.strip() # removes whitespaces in both end
    return sents

In [8]:
all_text=[]
labels= []
count=0
for label,file_name in zip(reader.categories(), reader.fileids()):
    sentences= sent_tokenize_whole_tweets(reader.raw(file_name)) # --> this should return a list of contents
    labels.extend([label for i in sentences[:-1]]) # son satırı sil
    #labels.remove(labels[-1])
    all_text.extend([i for i in sentences[:-1]]) # son satırı sil
    #all_text.remove(all_text[-1])
    #print('Dosya adı:'+file_name+'DENEMEE')
#print(len(sentences))
print(len(labels))
print(len(all_text))
# Now, we have all tweets in all_text list!

22347
22347


In [9]:
import numpy as np

all_text= np.array(all_text)


In [10]:
labels= np.array(labels)

In [11]:
# WITH UNIGRAM COUNT VECTORIZER:
import numpy as np
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC

from sklearn.model_selection import StratifiedKFold 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score
# The folds are made by preserving the percentage of samples for each class.
# use the original data, all_text!

list_classifier= [MultinomialNB(), LinearSVC()]

for clf in list_classifier:
    print(clf)
    skf = StratifiedKFold(n_splits=5,shuffle=True, random_state=123)
    scores= []
    i= 1
    for train_index, test_index in skf.split(all_text, labels):
        X_train, y_train = all_text[train_index], labels[train_index] 
        X_test, y_test = all_text[test_index], labels[test_index]

        train_vectorizer = CountVectorizer()
        X_train = train_vectorizer.fit_transform(X_train)
        train_vocab= train_vectorizer.vocabulary_   
        test_vectorizer = CountVectorizer(vocabulary=train_vocab) # train_vocab is being used!
        X_test = test_vectorizer.fit_transform(X_test)
        clf.fit(X_train, y_train)
        sc= accuracy_score(y_test, clf.predict(X_test))
        scores.append(sc)
        print("fold "+ str(i)+ " accuracy: "+ str(sc))
        i+= 1
    print("Mean score: "+ str(np.mean(scores)))
    print(" ")

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)
fold 1 accuracy: 0.715977653631285
fold 2 accuracy: 0.7195885509838998
fold 3 accuracy: 0.7239991053455602
fold 4 accuracy: 0.7203313927451859
fold 5 accuracy: 0.7174546269325566
Mean score: 0.7194702659276976
 
LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)
fold 1 accuracy: 0.6712849162011173
fold 2 accuracy: 0.669051878354204
fold 3 accuracy: 0.6736747931111609
fold 4 accuracy: 0.670846394984326
fold 5 accuracy: 0.6668160430203899
Mean score: 0.6703348051342395
 


In [12]:
# WITH UNIGRAM TF-idf VECTORIZER:
import numpy as np
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC

from sklearn.model_selection import StratifiedKFold 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score
# The folds are made by preserving the percentage of samples for each class.
# use the original data, all_text!


from sklearn.feature_extraction.text import TfidfTransformer

list_classifier= [MultinomialNB(), LinearSVC()]

for clf in list_classifier:
    print(clf)
    skf = StratifiedKFold(n_splits=5,shuffle=True, random_state=123)
    #all_text= np.array(all_text)
    #labels= np.array(labels)
    scores= []
    i= 1
    for train_index, test_index in skf.split(all_text, labels):
        X_train, y_train = all_text[train_index], labels[train_index] 
        X_test, y_test = all_text[test_index], labels[test_index]

        train_vectorizer = CountVectorizer()
        X_train = train_vectorizer.fit_transform(X_train)
        transformer_tf= TfidfTransformer(use_idf= True, norm= 'l2', smooth_idf=True) 
        # if normalize None, it is the same as CountVect
        # l2 normalization: like percentage of values
        X_train= transformer_tf.fit_transform(X_train)
           
        train_vocab= train_vectorizer.vocabulary_   
        test_vectorizer = CountVectorizer(vocabulary=train_vocab)
        X_test = test_vectorizer.fit_transform(X_test)
        X_test= transformer_tf.fit_transform(X_test)
        
        clf.fit(X_train, y_train)
        sc= accuracy_score(y_test, clf.predict(X_test))
        scores.append(sc)
        print("fold "+ str(i)+ " accuracy: "+ str(sc))
        i+= 1
    print("Mean score: "+ str(np.mean(scores)))
    print(" ")



MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)
fold 1 accuracy: 0.6451396648044693
fold 2 accuracy: 0.6435599284436494
fold 3 accuracy: 0.6445985238201745
fold 4 accuracy: 0.6560680698611733
fold 5 accuracy: 0.6446336544924939
Mean score: 0.646799968284392
 
LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)
fold 1 accuracy: 0.76268156424581
fold 2 accuracy: 0.7649821109123435
fold 3 accuracy: 0.7711921270409304
fold 4 accuracy: 0.7662337662337663
fold 5 accuracy: 0.773022630517589
Mean score: 0.7676224397900878
 


### Look at the following small examples to ensure that everything works as expected:

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
>>> corpus = [
...     'This is the first document.',
...     'This document is the second document.',
...     'And this is the third one.',
...     'Is this the first document?',
... ]
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(corpus)
print(vectorizer.get_feature_names())
print(X.toarray())  

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer
transformer_tf= TfidfTransformer(use_idf= False, norm=None)

X_tf= transformer_tf.fit_transform(X)
print(X_tf.toarray())  

In [ ]:
transformer_tfidf= TfidfTransformer(use_idf= True, norm=None)

X_tfidf= transformer_tfidf.fit_transform(X)
print(X_tfidf.toarray()) 

In [ ]:
import numpy as np
print((X.toarray()).shape[0])
print((X.toarray()).shape[1])
X_binary= np.zeros(((X.toarray()).shape[0],(X.toarray()).shape[1]))
print(X_binary.shape)
for i in range(len(X.toarray())):
    for j in range(len(X.toarray()[i])):
        if X.toarray()[i,j]>0:
            X_binary[i,j]= True
        else:
            X_binary[i,j]= False
print(X_binary)